In [1]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'

/mnt/nfs-data/users/anhvth5/projects/guidance


In [7]:
from guidance import *

In [6]:

models.QwenChat()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
